<a href="https://colab.research.google.com/github/run-llama/llamacloud-demo/blob/main/examples/getting_started_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started: Building a Chat Engine with LlamaCloud

In this notebook, we show you how to build a multi-step chat engine over a LlamaCloud index over your data.

Out chat engines allow you to turn a RAG pipeline into a conversational chat interface. During each turn, we maintain conversation history and use that retrieve context and synthesize over the relevant chat history.

**How the `CondensePlusContextChatEngine` works**
* First condense a conversation and latest user message to a standalone question
* Then build a context for the standalone question from a retriever,
* Then pass the context along with prompt and user message to LLM to generate a response.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
%pip install llama-index-llms-openai
%pip install llama-index
%pip install llama-index-indices-llama-cloud

## Build LlamaCloud Index

The LlamaCloud index is built over the 2021 Lyft and Uber 10K documents.

To create the index, follow the instructions:
1. You can download them here ([Uber 10K](https://www.dropbox.com/s/te0a2w227v27iag/uber_2021.pdf?dl=1), [Lyft 10K](https://www.dropbox.com/s/qctkz6nxhm0y5qe/lyft_2021.pdf?dl=1))
2. Follow instructions on `https://cloud.llamaindex.ai/` to signup for an account. Create a pipeline by uploading these documents.

In [18]:
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex

index = LlamaCloudIndex(
  name="<index_name>", 
  project_name="<project_name>",
  api_key="llx-"
)

In [19]:
retriever = index.as_retriever()

## Build Chat Engine

Define a chat engine wrapper around the defined LlamaCloud index.

Since the context retrieved can take up a large amount of the available LLM context, let's ensure we configure a smaller limit to the chat history!

In [20]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI


llm = OpenAI(model="gpt-3.5-turbo")

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

chat_engine = CondensePlusContextChatEngine.from_defaults(
    index.as_retriever(),
    memory=memory,
    llm=llm,
    context_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about financial reports for Uber and Lyft."
        "Here are the relevant documents for the context:\n"
        "{context_str}"
        "\nInstruction: Use the previous chat history, or the context above, to interact and help the user."
    ),
    verbose=False,
)

Chat with your data

In [21]:
response = chat_engine.chat("What are the Monthly Active Platform Consumers in 2021?")

In [22]:
print(len(response.source_nodes))
print(str(response))

3
In 2021, the Monthly Active Platform Consumers (MAPCs) for Uber were 118 million. This represents a 27% increase from the previous year. MAPCs are the number of unique consumers who completed a Mobility or New Mobility ride or received a Delivery order on Uber's platform at least once in a given month.


Ask a follow up question

In [23]:
response_2 = chat_engine.chat("What about 2020?")

In [24]:
print(response_2)

In 2020, the Monthly Active Platform Consumers (MAPCs) for Uber were 93 million. This shows a 27% increase to 118 million in 2021. MAPCs are an important metric for Uber as they indicate the adoption of their platform and the frequency of transactions by consumers.


Reset conversation state

In [25]:
chat_engine.reset()

In [26]:
response = chat_engine.chat("Hello! What do you know?")

In [27]:
print(response)

Hello! I'm here to help you with any questions or information you need. Is there anything specific you would like to know or talk about today?


## Streaming Support

In [28]:
chat_engine = CondensePlusContextChatEngine.from_defaults(
    index.as_retriever(),
    chat_mode="condense_plus_context",
    context_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about financial reports for Uber and Lyft."
        "Here are the relevant documents for the context:\n"
        "{context_str}"
        "\nInstruction: Based on the above documents, provide a detailed answer for the user question below."
    ),
)

In [29]:
response = chat_engine.stream_chat("What are the assets/liabilities for Lyft?")
for token in response.response_gen:
    print(token, end="")

Based on the financial report provided for Lyft, here is a breakdown of the assets and liabilities for the company:

Assets:
1. Receivables: The receivable balance primarily consists of amounts due from Enterprise Users and was $196.2 million, $104.7 million, and $120.0 million as of December 31, 2021, 2020, and 2019, respectively.
2. Allowance for Credit Losses: The allowance for credit losses, which reflects the Company's estimate of expected credit losses inherent in the enterprise and trade receivables balance, was $9.3 million, $15.2 million, and $6.2 million as of December 31, 2021, 2020, and 2019, respectively.

Liabilities:
1. Prepaid Expenses and Other Current Assets: Uncollected fees owed for completed transactions on the Lyft Platform are included in prepaid expenses and other current assets on the consolidated balance sheets.
2. Accrued and Other Current Liabilities: The portion of the fare receivable to be remitted to drivers is included in accrued and other current liabil